# Introduction
\[_In case you’re unable to see the atoti visualizations in GitHub, try viewing the notebook in [nbviewer](https://nbviewer.org/github/atoti/notebooks/blob/master/notebooks/airline-industry/main.ipynb)._]

## OLAP operations with atoti for use case from Airline Industry
From the vast domain of the aviation sector, we present a use case pertaining to performance of the airline industry during the covid-19 pandemic. The notebook illustrates the smooth blending of olap operations with atoti to simulate/model multiple *“what if”* scenarios easily, to gain crucial insights.

## Dataset used:
The simulated dataset has the three dimensions as :-
* airlines - Contains some important information pertaining to different  airlines, types of aircrafts, categories of booking class, seats availability w.r.t booking class, daily departures  among others.
* routes - The routes operated by each airline and distance between the source and destination in kilometers for each airline etc.
* date - The occurrence of real world events such as booking a ticket can be measured at different levels of hierarchy such as year, month, date, etc.

In addition to that we have the fact table(which has the measures) 
* salesdata -  Here all details containing the tickets sold by a particular airline along specific routes are filled which can be used to get insights about some business performance metrics such as revenue generated by a specific airline, revenue generated along specific routes, revenue generated with respect to a particular booking category, etc.

* The use case considers seven airlines and 24 routes for each of the seven airlines.

## Importing Libraries

In [ ]:
# Import python libraries
import datetime

import pandas as pd

# Import atoti library
import atoti as tt

# Part I - Loading, joining the data and creating cubes

## Load sales data

In [ ]:
# creating a session
session = tt.Session(user_content_storage= "./content")

In [ ]:
session.link()

In [ ]:
# loading salesdata
salesdata = pd.read_csv("data/salesdata.csv")
salesdata_table = session.read_pandas(
    salesdata,
    keys=["SalesId", "Date", "RouteId"],
    table_name="salesdata_table",
    types={
        "SalesId": tt.type.STRING,
        "Date": tt.type.STRING,
        "RouteId": tt.type.STRING,
    },
)
salesdata_table.head(8)

In [ ]:
# loading airlines dataset to skip the index column and then store it in atoti store

airline_df = pd.read_csv("data/airlines.csv", index_col=0)
airlines_table = session.read_pandas(
    airline_df, keys=["AirlineId"], table_name="airlines_table"
)
airline_df.head()

In [ ]:
# loading routes dataset

routes_table = session.read_csv("data/routes.csv", keys=["RouteId"])
routes_table.head()

In [ ]:
# loading dates dataset
df = pd.read_csv("data/date.csv")
df = df.astype(str)
dates_table = session.read_pandas(df, keys=["Date"], table_name="dates_table")
dates_table.head()

## Join the data tables

In [ ]:
# Now we join the newly created stores to the sales store

salesdata_table.join(airlines_table, mapping={"AirlineId": "AirlineId"})

salesdata_table.join(routes_table, mapping={"RouteId": "RouteId"})

salesdata_table.join(dates_table, mapping={"Date": "Date"})

## Creating the cube

In [ ]:
cube = session.create_cube(salesdata_table)

In [ ]:
session.tables.schema

In [ ]:
# defining measures, level and hierarchy
m = cube.measures
l = cube.levels
h = cube.hierarchies

## Defining New Custom Measure

As we saw in the salesdata table we have multiple active airlines for a day, but we may need daily aggregate revenue.So we will define a new measure

In [ ]:
m["Total Daily Revenue"] = tt.agg.sum(
    tt.value(salesdata_table["Total"]),
    scope=tt.scope.origin(l["Date"]),
)

# PART II: Visualizations

We Present the following insights by implementing olap operations (slice, dice and drill down) with atoti.

## Computing yearly and quarterly Revenue

The first insight visualised through pie-chart clearly shows that covid-19 had a negative impact on the airline industry as we see fall in revenue for year 2021 as compared to 2020. 

In [ ]:
session.visualize()

Furthermore if we see the performance of year 2020 across all four quarters we see that quarter three was the one which was negatively impacted as compared to other quarters.

In [ ]:
session.visualize()

## Computing quarterly revenue for four different airlines

This insight presents a comparative analysis of four different airlines that were active for the year 2020 and the revenue generated by each airline in these four quarters. We see that Allegiant Airlines business was negatively impacted in quarter 4. This can be used to simulate drill down operation as year → quarter with aggregating across routes.

In [ ]:
session.visualize()

## Computing quarterly revenue for four different airlines and drilling down across routes

Here we again have a comparative analysis of the above four airlines but now the revenue generated is shown along each specific route that the airlines were active in for all quarters.


In [ ]:
session.visualize()

## Computing quarterly revenue for a single airlines across routes by slicing

We further slice the cube for “American Airlines” to show the total revenue across the four quarters along 12 distinct routes.

In [ ]:
session.visualize("American Airlines")

## Computing revenue for a month by booking classes

This insight shows the revenue generated by a particular booking class. We consider three categories such as business class, premium economy and economy.
This is a dice operation as we dice it for year (2020), month (May) and Route(route1) for a specific airline(American Airline).

In [ ]:
salesdata.head()

In [ ]:
airlines_table.head()

In [ ]:
session.visualize()

## Visualizing the info in a dashboard 

The various scenarious and simulations can be simultaneously compared using a dashbard in atoti. 
These dashoboards are highly customisable and can be designed using the various widgets as per the business objective.
These can be saved and shared silpky by sharing a URL.
Click on the link below to explore a sample dashboard we created!!

In [ ]:
session.link(path="#/dashboard/088")

## Conclusion:

Such insights from the aviation industry can help the airlines for sales and marketing, exciting offers to attract customers post COVID-19 restrictions in travel, and generate any additional insights that could benefit their business as well their customer services. 

For more interesting notebooks, check out the [atoti gallary](https://github.com/atoti/notebooks)

# _ _The end_ _